In [1]:
import os
import timeit
import asyncio

from aiohttp import ClientSession

from web3.providers.base import JSONBaseProvider
from web3.providers import HTTPProvider
from web3 import Web3

In [2]:
# ipykernels have asyncio always running in the back. need to do this for other asyncios in the backend
import nest_asyncio
nest_asyncio.apply()

In [3]:
# synchronously request receipts for given transactions
def sync_receipts(web3, transactions):
    for tran in transactions:
        web3.eth.getTransactionReceipt(tran)

In [4]:
# asynchronous JSON RPC API request
async def async_make_request(session, url, method, params):
    base_provider = JSONBaseProvider()
    request_data = base_provider.encode_rpc_request(method, params)
    async with session.post(url, data=request_data,
                        headers={'Content-Type': 'application/json'}) as response:
        content = await response.read()
    response = base_provider.decode_rpc_response(content)
    return response

In [5]:
async def run(node_address, transactions):
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
        for tran in transactions:
            task = asyncio.ensure_future(async_make_request(session, node_address,
                                                            'eth_getTransactionReceipt',[tran.hex()]))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)

    return responses

In [6]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
eth_node_address = f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"
web3 = Web3(HTTPProvider(eth_node_address))

In [7]:
block = web3.eth.getBlock(web3.eth.blockNumber)
transactions = block['transactions']

In [8]:
transactions

[HexBytes('0x493a2b2a86677f8c6174ccfb6f51847f9c6ac80d64af260f0cc353efeb7f1e78'),
 HexBytes('0x0037ebfe7b3c9e7d42e54bb3b1a62b3176601d37652bfd7e675b5cebbc99f81e'),
 HexBytes('0xa5b96e0776e6f671946a7797df80ff7356283fc410b0c0eb4d62ac6f3699ac24'),
 HexBytes('0x135968cd3bdf4e2c9340eba7c5a43871b9caa20e781d3c8f323403eaa2268029'),
 HexBytes('0xa35bf7aa01fe2df15cb3553522c7414d7470080216aafb4e9928e396969a02b1'),
 HexBytes('0x3a2d8d4dcb4474cf6eefc40708084d569651665a4579b6afbbb0084fc9f0da90'),
 HexBytes('0x335ca7f786a73a8addb24859d8d81520ec6e102f91f5c4d3c42e5ec38ae5c14d'),
 HexBytes('0x1a1340b29ac1539b629031e12322f25921cbf28c0a2c68477d1ee9efc0aec702'),
 HexBytes('0x85cacc28c6766b26a1609445a77763b32d063e96496e12f5331e18895421681d'),
 HexBytes('0x9fafe3a8ab89a644da651f5ecfee29a847acca5bb3dd1667b837618111169cd4'),
 HexBytes('0x2d1d5306efd76a5c898d8fd12d02df7cc7803845d2fd12bc333c41e44b0679c4'),
 HexBytes('0x823ff89ecce3236023598ec2aa51b9ddf1eda7eb910aea39dd378f084466440d'),
 HexBytes('0x4ca228b1cb63d95

In [9]:
start_time = timeit.default_timer()
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run(eth_node_address, transactions))
result = loop.run_until_complete(future)
print('async: {:.3f}s'.format(timeit.default_timer() - start_time))

async: 0.509s


In [10]:
result

[{'jsonrpc': '2.0',
  'id': 0,
  'result': {'transactionHash': '0x493a2b2a86677f8c6174ccfb6f51847f9c6ac80d64af260f0cc353efeb7f1e78',
   'blockHash': '0xe8a78f64219f7f2b9e3ba5b57120b557141516b14805e645b2664775bea7c918',
   'blockNumber': '0xcfc181',
   'contractAddress': None,
   'cumulativeGasUsed': '0x87f7',
   'effectiveGasPrice': '0x28479a8200',
   'from': '0x918800e018a0eeea672740f88a60091c7d327a79',
   'gasUsed': '0x87f7',
   'logs': [{'address': '0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce',
     'topics': ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
      '0x000000000000000000000000918800e018a0eeea672740f88a60091c7d327a79',
      '0x000000000000000000000000d1130d0353902a8dd0bf9167f9f58147402525b5'],
     'data': '0x00000000000000000000000000000000000000000014791dda64cc6bed87cc00',
     'blockNumber': '0xcfc181',
     'transactionHash': '0x493a2b2a86677f8c6174ccfb6f51847f9c6ac80d64af260f0cc353efeb7f1e78',
     'transactionIndex': '0x0',
     'blockHash': 

In [9]:
start_time = timeit.default_timer()
sync_receipts(web3, transactions)
print('sync: {:.3f}s'.format(timeit.default_timer() - start_time))

sync: 81.911s
